# Libraries

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize
from scipy.interpolate import CubicSpline, interp1d

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

from skimage import filters

import SimpleITK as sitk

from Gafchromic import GafchromicFilms


output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS:


# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray



# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
# Converts the RGB image to RGBA for display
#   
# @params:
#  array: image RGB array
#  sizex: x size of the image
#  sizey: y size of the image
def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba




# Reads all images and returns the median image
#   
# @params:
#  array: image RGB array
#  sizex: x size of the image
#  sizey: y size of the image
def medianImage(path, filename, firstNb, fileExtension, nbOfImgs):

    img = sitk.ReadImage(path+filename+str(firstNb)+fileExtension)

    size = (sitk.GetArrayFromImage(img).shape[0], 
            sitk.GetArrayFromImage(img).shape[1], 
            sitk.GetArrayFromImage(img).shape[2], 
            nbOfImgs)
    
    imgs = np.zeros(size)
    
    for i in range(nbOfImgs):
        img = sitk.ReadImage(path+filename+str(firstNb+i)+fileExtension)
        imgs[:,:,:,i] = sitk.GetArrayFromImage(img)

    medianImg = np.median(imgs, axis=3)

    return medianImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    
    
    
# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def dispImgAndProf(img, col, line, plotwidth=450, title='dose image 1', colorprofile='firebrick'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img.shape[0]/img.shape[1]), 
                title=title, toolbar_location="above")
    p1.image(image=[img], x=0, y=0, dw=img.shape[1], dh=img.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img[line,:]), 1)
    p3.line(x3, img[line,:], line_width=2, line_color=colorprofile, legend_label=title)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img[:,col]), 1)
    p4.line(x4, img[:, col], line_width=3, line_color=colorprofile, legend_label=title)

    grid = gridplot([[p3, p4]])

    show(p1)
    show(grid)
    

# Computes coefficients

In [6]:
# READS THE IMAGES:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers

# Version généralisée: on doit pouvoir mettre autant d'images que l'on souhaite
# Peut etre faut-il renseigner les centres ? ou decomposer le champ de vue en 
# régions égales en taille?


# Inputs:
m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/corrLaterale/'

m_fileName = 'pos'

m_firstPosNb = 1
m_nbOfPos = 10

m_firstImgNb = 1
m_nbOfImgsPerPos = 1

m_fileExtension = ".tif"

m_nbRoisPerPos = 4
m_roiCenters = [35, 90, 140, 200]
m_roiHeight = 30
m_roiWidth = 16

m_dimViewer = 900
m_subfactor = 10

visu = False
m_beginRight = True


# Reading the images and creates an image having all positions:

# Reading first image:
img = medianImage(m_path, m_fileName+str(m_firstPosNb)+'_0', m_firstImgNb, m_fileExtension, m_nbOfImgsPerPos)
img = subSampleRGBArray(img, m_subfactor)

# creates the output image for calibration
calibImg = np.zeros(img.shape)

# variables to be used for cropping the images:
m_step = img.shape[1] / m_nbOfPos
m_layerPos = int(m_step/2-1)
if m_beginRight: 
    m_layerPos = (img.shape[1]-1) - m_layerPos
m_halfwidth = int(m_step/2-1)

# cropping first image and records it in the calibration image:
calibImg[:,m_layerPos-m_halfwidth:m_layerPos+m_halfwidth,:] = img[:,m_layerPos-m_halfwidth:m_layerPos+m_halfwidth,:]

# for loop to read all other images:
for i in range(1, m_nbOfPos):
    img = medianImage(m_path, m_fileName+str(m_firstPosNb+i)+'_0', m_firstImgNb, m_fileExtension, m_nbOfImgsPerPos)
    img = subSampleRGBArray(img, m_subfactor)
    
    m_layerPos = int((i+0.5)*m_step-1)
    if m_beginRight:
        m_layerPos = (img.shape[1]-1) - m_layerPos

    calibImg[:,m_layerPos-m_halfwidth:m_layerPos+m_halfwidth,:] = img[:,m_layerPos-m_halfwidth:m_layerPos+m_halfwidth,:]




# Visualization:
if visu:
    
    # draws the image :
    p = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*calibImg.shape[0]/calibImg.shape[1]), 
                   title='Calibration image', toolbar_location="above")
    p.image(image=[calibImg[:,:,0]], x=0, y=0, dw=calibImg.shape[1], dh=calibImg.shape[0], palette="Greys256")

    # draws ROIs used for calibration:
    for i in range(m_nbOfPos):
        m_layerPos = int((i+0.5)*m_step-1)
        if m_beginRight:
            m_layerPos = (img.shape[1]-1) - m_layerPos
        for j in range(len(m_roiCenters)):
            p.rect(x=m_layerPos, 
                   y=m_roiCenters[j], 
                   width=m_roiWidth,
                   height=m_roiHeight,
                   fill_alpha=0,
                   line_color="firebrick",
                   line_alpha=1
              )

    show(p)
    

    
print('   >>> iMAGES rEAD !')

   >>> iMAGES rEAD !


In [13]:
# CORRECTION FACTORS DETERMINATION
#   Correction Factors calculated in Lewis et al. Med Phys january 2015

visu = False

def func(x, a, b):
    return a + b*x


# Postioning array: (where the coefficients are calculated) in mm
#  The films are now scanned in 254ppp and resized to 25.4ppp which means 1pix/mm
#  This code only works in this case (1pix/mm)
#  It should be changed in future implementations!
m_posArr = []
for i in range(m_nbOfPos):
    m_posArr.append(int((i+0.5)*m_step-1))


# Center grey values (RGB) calculation:
vr_center = []
vg_center = []
vb_center = []

roiHalfWidth = int(m_roiWidth/2)
roiHalfHeight = int(m_roiHeight/2)

if m_nbOfPos%2 == 0:
    m_layerPos1 = int((m_nbOfPos/2+0.5)*m_step-1)
    if m_beginRight:
        m_layerPos1 = (img.shape[1]-1) - m_layerPos1

    m_layerPos2 = int((m_nbOfPos/2-0.5)*m_step-1)
    if m_beginRight:
        m_layerPos2 = (img.shape[1]-1) - m_layerPos2
    
    vr1 = []
    vg1 = []
    vb1 = []
    for i in range(m_nbRoisPerPos):
        vr1.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos1-roiHalfWidth:m_layerPos1+roiHalfWidth, 0]))
        vg1.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos1-roiHalfWidth:m_layerPos1+roiHalfWidth, 1]))
        vb1.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos1-roiHalfWidth:m_layerPos1+roiHalfWidth, 2]))

    vr2 = []
    vg2 = []
    vb2 = []
    for i in range(m_nbRoisPerPos):
        vr2.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos2-roiHalfWidth:m_layerPos2+roiHalfWidth, 0]))
        vg2.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos2-roiHalfWidth:m_layerPos2+roiHalfWidth, 1]))
        vb2.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos2-roiHalfWidth:m_layerPos2+roiHalfWidth, 2]))

    for i in range(m_nbRoisPerPos):
        vr_center.append((vr1[i]+vr2[i])/2)
        vg_center.append((vg1[i]+vg2[i])/2)
        vb_center.append((vb1[i]+vb2[i])/2)
    
else:
    m_layerPos = int((m_nbOfPos/2)*m_step-1)
    if m_beginRight:
        m_layerPos = (img.shape[1]-1) - m_layerPos1

    for i in range(m_nbRoisPerPos):
        vr_center.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 0]))
        vg_center.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 1]))
        vb_center.append(np.mean(calibImg[m_roiCenters[i]-roiHalfHeight:m_roiCenters[i]+roiHalfHeight,
                               m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 2]))


# finding the parameters A and B for each channel and position:
Ar = []
Br = []
Ag = []
Bg = []
Ab = []
Bb = []

for i in range(m_nbOfPos):
    vr = []
    vg = []
    vb = []
    
    m_layerPos = int((i+0.5)*m_step-1)
    if m_beginRight:  # is it necessary? needs to be checked...
        m_layerPos = (img.shape[1]-1) - m_layerPos
    
    for j in range(m_nbRoisPerPos):
        vr.append(np.mean(calibImg[m_roiCenters[j]-roiHalfHeight:m_roiCenters[j]+roiHalfHeight,
                           m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 0]))
        vg.append(np.mean(calibImg[m_roiCenters[j]-roiHalfHeight:m_roiCenters[j]+roiHalfHeight,
                           m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 1]))
        vb.append(np.mean(calibImg[m_roiCenters[j]-roiHalfHeight:m_roiCenters[j]+roiHalfHeight,
                           m_layerPos-roiHalfWidth:m_layerPos+roiHalfWidth, 2]))

    params, cov = optimize.curve_fit(func, vr, vr_center)
    Ar.append(params[0])
    Br.append(params[1])

    params, cov = optimize.curve_fit(func, vg, vg_center)
    Ag.append(params[0])
    Bg.append(params[1])

    params, cov = optimize.curve_fit(func, vb, vb_center)
    Ab.append(params[0])
    Bb.append(params[1])

if m_beginRight:
    Ar = np.flip(Ar)
    Ag = np.flip(Ag)
    Ab = np.flip(Ab)
    Br = np.flip(Br)
    Bg = np.flip(Bg)
    Bb = np.flip(Bb)
    
    
# data visualization:
signSize = 5
alpha = 0.5 

if visu:
    p1 = figure(plot_width=700, plot_height=500, title="A values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p1.line(m_posArr, Ar, line_width=2, color='firebrick')
    p1.line(m_posArr, Ag, line_width=2, color='darkgreen')
    p1.line(m_posArr, Ab, line_width=2, color='darkblue')
    
    p2 = figure(plot_width=700, plot_height=500, title="B values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient B')
    p2.line(m_posArr, Br, line_width=2, color='firebrick')
    p2.line(m_posArr, Bg, line_width=2, color='darkgreen')
    p2.line(m_posArr, Bb, line_width=2, color='darkblue')

    grid = gridplot([[p1], [p2]])
    show(grid)


print('   >>> Coefficients Calculated !')

   >>> Coefficients Calculated !


In [19]:
# CORRECTION MODELS FITTING: 


# To visualize results:
visu = False


fAr_lin = interp1d(m_posArr, Ar, kind='linear')  # kind = cubic, linear, ...
fAg_lin = interp1d(m_posArr, Ag, kind='linear')
fAb_lin = interp1d(m_posArr, Ab, kind='linear')

fBr_lin = interp1d(m_posArr, Br, kind='linear')
fBg_lin = interp1d(m_posArr, Bg, kind='linear')
fBb_lin = interp1d(m_posArr, Bb, kind='linear')


if visu:
    x = m_posArr
    xnew = np.arange(int(np.min(x)), int(np.max(x)))
    
    p1 = figure(plot_width=300, plot_height=300, title="Ar values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p1.circle(x, Ar, size=5, color='firebrick', alpha=0.5)
    p1.line(xnew, fAr_lin(xnew), line_width=1, color='firebrick', line_dash='dashed')

    p2 = figure(plot_width=300, plot_height=300, title="Ag values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p2.circle(x, Ag, size=5, color='darkgreen', alpha=0.5)
    p2.line(xnew, fAg_lin(xnew), line_width=1, color='darkgreen', line_dash='dashed')

    p3 = figure(plot_width=300, plot_height=300, title="Ab values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p3.circle(x, Ab, size=5, color='darkblue', alpha=0.5)
    p3.line(xnew, fAb_lin(xnew), line_width=1, color='darkblue', line_dash='dashed')

#     p1.line(x, Ag, line_width=2, color='darkgreen')
#     p1.line(x, Ab, line_width=2, color='darkblue')
    
    p4 = figure(plot_width=300, plot_height=300, title="Br values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p4.circle(x, Br, size=5, color='firebrick', alpha=0.5)
    p4.line(xnew, fBr_lin(xnew), line_width=1, color='firebrick', line_dash='dashed')

    p5 = figure(plot_width=300, plot_height=300, title="Bg values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p5.circle(x, Bg, size=5, color='darkgreen', alpha=0.5)
    p5.line(xnew, fBg_lin(xnew), line_width=1, color='darkgreen', line_dash='dashed')

    p6 = figure(plot_width=300, plot_height=300, title="Bb values", toolbar_location="above",
               x_axis_label='Lateral position (mm)', y_axis_label='Coefficient A')
    p6.circle(x, Bb, size=5, color='darkblue', alpha=0.5)
    p6.line(xnew, fBb_lin(xnew), line_width=1, color='darkblue', line_dash='dashed')

    grid1 = gridplot([[p1,p2,p3],[p4,p5,p6]])
    show(grid1)

print("   >>> mODELS dONE!")

   >>> mODELS dONE!


# Images Correction:

In [30]:
# IMAGE CORRECTION AND CONVERSION TO DOSE:


# Variables:
path = 'testRBGB/'
nbOfFiles = 1
firstNb = 1
filesName = "testRBGB_00"
fileExtension = ".tif"
splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data.txt'
pixsize = 1

# Reading the file:
try:
    g = GafchromicFilms(path+filesName, firstNb, nbOfFiles, fileExtension=fileExtension)
    g.subSampleDataArray(10)
    doseImg_noCorr = g.convertToDose_cubicSplineFit(splineFile, 1500)
    array = g.getArray()

    # correcting lateral response
    corrArray = np.zeros(array.shape)
    for i in range(array.shape[1]):
        posx = i*pixsize  # peut être faire plus précis ensuite..
        if posx<np.amin(m_posArr):
            posx = np.amin(m_posArr)
        elif posx>np.amax(m_posArr):
            posx = np.amax(m_posArr)
        
        _Ar = fAr_lin(posx)
        _Ag = fAg_lin(posx)
        _Ab = fAb_lin(posx)
        _Br = fBr_lin(posx)
        _Bg = fBg_lin(posx)
        _Bb = fBb_lin(posx)
        corrArray[:,i,0] = _Ar + _Br*array[:,i,0]
        corrArray[:,i,1] = _Ag + _Bg*array[:,i,1]
        corrArray[:,i,2] = _Ab + _Bb*array[:,i,2]
    
    g.setArray(corrArray)
    doseImg_corr = g.convertToDose_cubicSplineFit(splineFile, 1500)
except ValueError as err:
    print('Erreur: '+err)

    
compare2Imgs(doseImg_noCorr, doseImg_corr, 200, 220, plotwidth=450, 
             title1="no corr.", title2="corr.")